In [5]:
pip install keyboard


In [4]:
import cv2
import numpy as np
import os
import serial
import time
import tensorflow as tf
import keyboard
model = tf.keras.models.load_model(r'C:\Users\lenovo\Desktop\custom_vgg16_model.h5')
folder_path = r'E:\Trash'
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        os.remove(file_path)
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")
IMG_SIZE = 32
CATEGORIES = ['can', 'glass', 'paper', 'plastic']
def capture_image():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print("Cannot open camera")
        return None
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        return None
    image_path = os.path.join(folder_path, 'captured_image.jpg')
    cv2.imwrite(image_path, frame)

    cap.release()
    cv2.destroyAllWindows()

    return image_path
def clean_folder():
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")
try:
    arduino = serial.Serial(port='COM3', baudrate=9600, timeout=1)
    time.sleep(2)
except serial.SerialException as e:
    print(f"Error opening serial port: {e}")
    exit()
try:
    while True:
        trash_type = None
        while True:
            if keyboard.is_pressed('c'):
                trash_type = "can"
                break
            elif keyboard.is_pressed('g'):
                trash_type = "glass"
                break
            elif keyboard.is_pressed('p'):
                trash_type = "paper"
                break
            elif keyboard.is_pressed('l'):
                trash_type = "plastic"
                break
        while True:
            if arduino.in_waiting > 0:
                data = arduino.readline().decode('utf-8').strip()
                print(f"Received from Arduino: {data}")
                if data == "close":
                    print("Arduino sent 'close', capturing image...")
                    image_path = capture_image()
                    if image_path:
                        print(f"Classified as: {trash_type}")
                        arduino.write(f"{trash_type}\n".encode())
                        time.sleep(1)
                        clean_folder()
                        print("Image deleted, folder cleaned up.")
                    else:
                        print("Failed to capture image.")
                    break
        if data == "open":
            print("Arduino sent 'open', stopping the process...")
            continue
except KeyboardInterrupt:
    print("Program interrupted by user.")
finally:
    if 'arduino' in locals() and arduino.is_open:
        arduino.close()
        print("Serial connection closed.")


Received from Arduino: close
Arduino sent 'close', capturing image...
Classified as: can
Image deleted, folder cleaned up.
Received from Arduino: open
Received from Arduino: close
Arduino sent 'close', capturing image...
Classified as: paper
Image deleted, folder cleaned up.
Serial connection closed.


SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))